<a href="https://colab.research.google.com/github/AvigdorFeldman/Collab/blob/main/Cloud_Snake_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Installs**

In [5]:
# ============================
# INSTALLS
# ============================
!pip -q install nltk pymupdf requests
!pip install --upgrade google-genai
# ============================
# IMPORTS
# ============================
import re, math, os
import requests
from google.colab import ai
import fitz  # PyMuPDF
from collections import defaultdict

import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("punkt_tab") # Added this line to download the missing resource
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### **Index**

In [6]:
# !pip install openai
# import os
# from openai import OpenAI
# from google.colab import userdata

# # TEMP: set in this session (better than hard-coding in functions)
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API")
# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [7]:

# ============================
# GOOGLE DRIVE URL HELPERS
# ============================
def extract_drive_file_id(url: str):
    """
    Supports:
    - https://drive.google.com/file/d/<ID>/view
    - https://drive.google.com/open?id=<ID>
    - https://drive.google.com/uc?id=<ID>&export=download
    """
    m = re.search(r"/file/d/([^/]+)", url)
    if m:
        return m.group(1)
    m = re.search(r"[?&]id=([^&]+)", url)
    if m:
        return m.group(1)
    return None
import re
import requests

def extract_drive_file_id(url: str):
    m = re.search(r"/file/d/([^/]+)", url)
    if m:
        return m.group(1)
    m = re.search(r"[?&]id=([^&]+)", url)
    if m:
        return m.group(1)
    return None

def drive_direct_download_url(url: str):
    file_id = extract_drive_file_id(url)
    if not file_id:
        return url
    return f"https://drive.google.com/uc?export=download&id={file_id}"

def download_pdf_bytes(url: str, timeout=60):
    session = requests.Session()
    direct = drive_direct_download_url(url)

    r = session.get(direct, stream=True, timeout=timeout)
    r.raise_for_status()

    # If Drive returns HTML, try confirm token
    content_type = (r.headers.get("Content-Type") or "").lower()
    if "text/html" in content_type:
        confirm_token = None
        for k, v in session.cookies.items():
            if k.startswith("download_warning"):
                confirm_token = v
                break
        if confirm_token:
            r = session.get(direct + f"&confirm={confirm_token}", stream=True, timeout=timeout)
            r.raise_for_status()

    data = r.content
    if not data.startswith(b"%PDF"):
        raise ValueError("Downloaded content is not a PDF. Check sharing permissions (Anyone with the link can view).")
    return data

def download_pdf_bytes(url: str, timeout=60):
    """
    Downloads a PDF from:
    - direct PDF URL, or
    - Google Drive file URL (handles the confirm token when Drive warns about virus scan / large file)
    """
    session = requests.Session()
    direct = drive_direct_download_url(url)

    # First request
    r = session.get(direct, stream=True, timeout=timeout)
    r.raise_for_status()

    # Google Drive sometimes requires a confirm token (returns HTML)
    content_type = (r.headers.get("Content-Type") or "").lower()
    if "text/html" in content_type or "drive.google.com" in r.url:
        # try to find confirm token in cookies
        confirm_token = None
        for k, v in session.cookies.items():
            if k.startswith("download_warning"):
                confirm_token = v
                break

        if confirm_token:
            r = session.get(direct + f"&confirm={confirm_token}", stream=True, timeout=timeout)
            r.raise_for_status()
        # else: could still be HTML if permissions are not public

    data = r.content
    # quick sanity check: PDFs usually start with %PDF
    if not data.startswith(b"%PDF"):
        raise ValueError(
            "Downloaded content is not a PDF. "
            "Make sure the Drive file is shared as 'Anyone with the link can view'."
        )
    return data

# ============================
# PDF TEXT EXTRACTION
# ============================
def pdf_bytes_to_text(pdf_bytes: bytes, max_chars=20000):
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    parts = []
    for page in doc:
        parts.append(page.get_text("text"))
    text = re.sub(r"\s+", " ", " ".join(parts)).strip()
    return text[:max_chars]

# ============================
# LOAD DOCUMENTS FROM URLS
# ============================
def load_papers_from_urls(urls, max_chars=20000):
    documents = []
    kept_urls = []
    for url in urls:
        try:
            pdf_data = download_pdf_bytes(url)
            text = pdf_bytes_to_text(pdf_data, max_chars=max_chars)
            documents.append(text)
            kept_urls.append(url)
            print("Loaded:", url)
        except Exception as e:
            print("FAILED:", url, "|", str(e))
    return documents, kept_urls



In [8]:
# ============================
# PREPROCESSING
# ============================
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess(text: str):
    text = re.sub(r"[^a-zA-Z0-9 ]", " ", text)
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return tokens

In [9]:
def TFIDF(tf, df, n):
    return tf * math.log(n / df)
# MODULAR URL INDEX (keeps your TF/DF/build pattern)
# ============================

def TF_urls(documents, urls):
    """
    Returns:
      doc_term_counts: list of dicts, aligned with urls:
        doc_term_counts[i][term] = tf in document i
    """
    doc_term_counts = []
    for text in documents:
        counts = defaultdict(int)
        for term in preprocess(text):
            counts[term] += 1
        doc_term_counts.append(counts)
    return doc_term_counts

def DF_urls(doc_term_counts):
    """
    Returns:
      df[term] = number of documents containing term
    """
    df = defaultdict(int)
    for counts in doc_term_counts:
        for term in counts.keys():
            df[term] += 1
    return df

def build_url_list_index(doc_term_counts, urls):
    """
    Returns:
      index_urls[term] = [url1, url2, ...] (sorted)
    """
    index_urls = defaultdict(set)

    for i, counts in enumerate(doc_term_counts):
        url = urls[i]
        for term in counts.keys():              # presence is enough for URL list
            index_urls[term].add(url)

    return {term: sorted(list(u_set)) for term, u_set in index_urls.items()}

def build_stats_index(doc_term_counts, urls, df, n_docs):
    """
    Returns:
      stats[term][url] = {"count": tf, "rank": tfidf}
    """
    stats = defaultdict(dict)

    for i, counts in enumerate(doc_term_counts):
        url = urls[i]
        for term, tf in counts.items():
            stats[term][url] = {
                "count": tf,
                "rank": TFIDF(tf, df[term], n_docs)
            }

    return stats

def build_inverted_index_urls(documents, urls, with_tfidf=True):
    """
    Main builder (modular):
      - index_urls: term -> list of urls that contain the term
      - stats (optional): term -> url -> {count, rank}
      - df: term -> doc frequency
    """
    n_docs = len(documents)

    # 1) TF per document
    doc_term_counts = TF_urls(documents, urls)

    # 2) DF per term
    df = DF_urls(doc_term_counts)

    # 3) term -> urls list
    index_urls = build_url_list_index(doc_term_counts, urls)

    if not with_tfidf:
        return index_urls, None, df

    stats = build_stats_index(doc_term_counts, urls, df, n_docs)
    return index_urls, stats, df

def retrieve_urls(query, index_urls, stats=None, top_k=3):
    q_terms = preprocess(query)
    scores = defaultdict(float)
    for term in q_terms:
        if term not in index_urls:
            continue
        for url in index_urls[term]:
            if stats is None:
                scores[url] += 1.0  # simple matching
            else:
                # add rank if available, otherwise fallback to +1
                scores[url] += stats.get(term, {}).get(url, {}).get("rank", 1.0)

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return ranked

In [10]:
papers_urls = [
    "https://drive.google.com/file/d/11ANjTBB6MGLgBQFg4q25YChjDYPrxLzQ/view",
    "https://drive.google.com/file/d/1kgwtzK4TWKMFKnOxv8u2r1ad6z1wvLNf/view",
    "https://drive.google.com/file/d/14vugPbpa8AdA-t-Cwgsdh8tuUocrhoaq/view",
    "https://drive.google.com/file/d/1eDEw6frpO2aZKHzHrtjnxXiGjk-89pzv/view",
    "https://drive.google.com/file/d/1aSOVQ22W8jH3aRAa9RijJZZ0PM1DUj4-/view"
]

In [11]:
documents, urls = load_papers_from_urls(papers_urls, max_chars=200000)
print("Loaded papers:", len(documents))

Loaded: https://drive.google.com/file/d/11ANjTBB6MGLgBQFg4q25YChjDYPrxLzQ/view
Loaded: https://drive.google.com/file/d/1kgwtzK4TWKMFKnOxv8u2r1ad6z1wvLNf/view
Loaded: https://drive.google.com/file/d/14vugPbpa8AdA-t-Cwgsdh8tuUocrhoaq/view
Loaded: https://drive.google.com/file/d/1eDEw6frpO2aZKHzHrtjnxXiGjk-89pzv/view
Loaded: https://drive.google.com/file/d/1aSOVQ22W8jH3aRAa9RijJZZ0PM1DUj4-/view
Loaded papers: 5


In [12]:
index_urls, stats,_ = build_inverted_index_urls(documents, urls, with_tfidf=True)

### **RAG**

In [98]:
import gdown

file_id = "1XKOKgb6avr7axa0cK_xamyieHh70KQGQIvqZhGzj_sY"
# Modified URL to explicitly request plain text format for Google Docs
url = f"https://docs.google.com/document/d/{file_id}/export?format=txt"
output = "api_key.txt"

gdown.download(url, output, quiet=False)

with open(output, "r") as f:
    key = f.read().strip().replace('\ufeff', '') # Remove BOM character


/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/document/d/1XKOKgb6avr7axa0cK_xamyieHh70KQGQIvqZhGzj_sY/export?format=txt
To: /content/api_key.txt
42.0B [00:00, 65.6kB/s]


In [18]:
# # ============================
# # RAG GENERATION WITH OPENAI (URL-based)
# # ============================
# url_to_doc = {u: i for i, u in enumerate(urls)}
# def rag_answer(query, index_urls, documents, urls, stats=None, top_k=3):
#     """
#     index_urls: term -> [url1, url2, ...]
#     documents : list of extracted text, aligned with urls
#     urls      : list of urls, same length/order as documents
#     stats     : optional term -> url -> {count, rank} for TF-IDF scoring
#     """
#     ranked = retrieve_urls(query, index_urls, stats=stats, top_k=top_k)  # [(url, score), ...]

#     # build context from the matching URLs
#     context_parts = []
#     for url, score in ranked:
#         doc_idx = url_to_doc[url]
#         text = documents[doc_idx]
#         context_parts.append(f"URL: {url}\nScore: {score:.4f}\nContent:\n{text[:2000]}")

#     context = "\n\n---\n\n".join(context_parts)

#     prompt = f"""
# Answer the question using ONLY the following academic paper excerpts.

# {context}

# Question: {query}

# Rules:
# - If the context is insufficient, say you don't have enough information from the provided papers.
# - Do not use outside knowledge.
# """

#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": prompt}]
#     )

#     return response.choices[0].message.content


In [14]:
# --- Global RAG State (for UI inspection) ---
rag_context_content = ""
rag_ranked_urls_list = []

In [15]:
url_to_doc = {u: i for i, u in enumerate(urls)}

In [78]:
import google.generativeai as genai

genai.configure(api_key=key, transport="rest")
model = genai.GenerativeModel("gemini-2.5-flash")

In [90]:
def rag_answer(
    query,
    index_urls,
    documents,
    urls,
    top_k=3,
    stats=None,
    stream=False
):
    """
    FULL RAG PIPELINE:
    Retrieval → Context → Prompt → LLM → Answer
    """

    global rag_context_content, rag_ranked_urls_list, url_to_doc

    rag_context_content = ""
    rag_ranked_urls_list = []

    # --- 1. Retrieval ---
    ranked = retrieve_urls(query, index_urls, stats=stats, top_k=top_k)
    rag_ranked_urls_list = ranked

    if not ranked:
        return {
            "answer": "I don't have enough information from the provided papers.",
            "error": True # Changed this to True for error conditions
        }

    # --- 2. Context ---
    context_parts = []
    for url, score in ranked:
        doc_idx = url_to_doc.get(url)
        if doc_idx is not None:
            context_parts.append(
                f"URL: {url}\nScore: {score:.4f}\nContent:\n{documents[doc_idx][:1000]}"
            )

    context = "\n\n---\n\n".join(context_parts)
    rag_context_content = context

    # --- 3. Prompt ---
    prompt = f"""
You are an expert ecological research assistant.
Answer ONLY using the provided academic paper excerpts.
If the information is insufficient, say so clearly.

--- CONTEXT ---
{context}

--- QUESTION ---
{query}
"""
    model_name = 'gemini-2.5-flash'
    # --- 4. LLM CALL ---
    if model == None:
      if stream:
        print(1)
        return { # Wrap the generator in a dictionary with a 'stream' key
            "stream": ai.generate_text(
                prompt=prompt,
                model_name=model_name,
                stream=True
            ),
            "error": False # No error, just streaming
        }

      response = ai.generate_text(
          prompt=prompt,
          model_name=model_name
      )

      return {
          "answer": response,
          "error": False
      }
    else:
      print(2)
      response = model.generate_content(prompt)
      plain_text = response.text
      if stream:
        return { # Wrap the generator in a dictionary with a 'stream' key
            "stream": plain_text,
            "error": False # No error, just streaming
        }
      else:

        return {"answer": plain_text, "error": False}

### **Screen 1: Analayze Plant Diseases**

In [20]:
# First Screen Upload Photos with AI (Hugging Face Local Only)

# ============================================================================
# PART 0: IMPORTS AND INITIAL SETUP
# ============================================================================
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image as PILImage
from io import BytesIO
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, Image, clear_output, Javascript
from base64 import b64decode
import io
import os

# Note: eval_js is kept as it's part of the camera capture JS logic.

In [21]:
# --- Global Variables for AI Clients and Models ---
loaded_model = None
loaded_processor = None
label_names = []

MODEL_HUB_NAME = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

# --- 1. Load Label Names ---
try:
    from datasets import load_dataset
    dataset = load_dataset("BrandonFors/Plant-Diseases-PlantVillage-Dataset", split="train")
    label_names = dataset.features['label'].names
except Exception:
    label_names = [f"Class {i}" for i in range(38)]

# --- 2. Load MobileNet (Local Hugging Face Model) ---
try:
    loaded_processor = AutoImageProcessor.from_pretrained(MODEL_HUB_NAME)
    loaded_model = AutoModelForImageClassification.from_pretrained(
        MODEL_HUB_NAME, num_labels=len(label_names), ignore_mismatched_sizes=True
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_model.to(device)
    print(f"✅ Local MobileNet loaded successfully on {device}.")
except Exception as e:
    loaded_model = None
    print(f"❌ Error loading local MobileNet: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/170M [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:30: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/9.34M [00:00<?, ?B/s]

✅ Local MobileNet loaded successfully on cpu.


In [22]:
def predict_disease_from_bytes(image_bytes, model, processor, label_names, top_k=3):
    if model is None:
        return [("Error: Local AI Model is not loaded.", 0)]
    try:
        image = PILImage.open(BytesIO(image_bytes)).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        device = model.device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
        top_probs, top_indices = torch.topk(probabilities, k=top_k)
        return [(label_names[idx.item()], prob.item()) for prob, idx in zip(top_probs, top_indices)]
    except Exception as e:
        return [("Error processing image: " + str(e), 0)]

def analyze_image_unified(image_bytes):
    global loaded_model, loaded_processor, label_names
    if loaded_model is not None:
        results = predict_disease_from_bytes(image_bytes, loaded_model, loaded_processor, label_names)
        result_html = "<h4>MobileNet Plant Disease Prediction:</h4><ul>"
        for j, (disease, confidence) in enumerate(results):
            style = "font-weight: bold; color: #38761d;" if j == 0 else "color: #555;"
            result_html += f'<li><span style="{style}">{disease}:</span> {confidence:.2%}</li>'
        result_html += "</ul>"
        return result_html, "LOCAL_HF"
    return "Error: Model failed to load.", "ERROR"

def take_photo_to_bytes(quality=0.8):
    from google.colab.output import eval_js
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture Photo';
            capture.style.cssText = 'background-color: #6aa84f; color: white; border: none; padding: 10px 20px; border-radius: 5px; cursor: pointer; margin-top: 10px;';
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            div.appendChild(capture);
            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => capture.onclick = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth; canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    try:
        data = eval_js('takePhoto({})'.format(quality))
        return b64decode(data.split(',')[1])
    except:
        return None

In [23]:
# --- Global variables to store uploaded image data and current index ---
uploaded_images = []
current_image_index = 0

def setup_first_screen_ui():
    """Assembles and returns the UI for the first screen (Image Upload & Analysis)."""

    # --- 1. Global CSS Definitions (Adjusted for Local-only theme) ---
    # This CSS will be applied when the tab is displayed via the main display call
    global css_style
    css_style = """
    <style>
        .widget-vbox.main-container {
            background-color: #ffffff !important; border: 1px solid #e0e0e0 !important; border-radius: 10px !important;
            padding: 40px !important; margin: 20px auto !important;
            font-family: Arial, sans-serif !important; text-align: center !important; box-shadow: 0 4px 12px rgba(0,0,0,0.05);
        }
        body { background-color: #f7fff7 !important; }
        h3 { color: #38761d !important; font-weight: bold !important; margin-top: 10px !important; margin-bottom: 5px !important; }
        h4 { color: #38761d !important; margin-top: 15px !important; margin-bottom: 5px !important; }
        p.subtitle { color: #555 !important; font-size: 14px !important; margin-bottom: 30px !important; }
        .camera-icon-container { width: 80px; height: 80px; background-color: #e6f7e6; border-radius: 50%; display: flex; justify-content: center; align-items: center; margin: 0 auto 15px auto; border: 2px solid #6aa84f; }
        .camera-icon { font-size: 36px !important; color: #6aa84f !important; }
        .upload-icon { font-size: 36px !important; color: #6aa84f !important; margin-bottom: 10px !important; }
        .drop-zone-vbox { border: 2px dashed #93c47d !important; border-radius: 8px !important; padding: 40px 20px !important; background-color: #fafff7 !important; margin-bottom: 30px !important; }
        .drop-zone-text p { color: #555 !important; margin: 0 !important; line-height: 1.5 !important; }
        .widget-upload .btn.btn-success { background-color: #6aa84f !important; border-color: #6aa84f !important; color: white !important; padding: 8px 20px !important; font-weight: bold !important; margin-top: 15px !important; }
        .widget-upload .btn.btn-success:hover { background-color: #38761d !important; border-color: #38761d !important; }
        .image-display-output { margin-top: 20px; margin-bottom: 20px; border: 1px solid #ddd; border-radius: 5px; padding: 10px; background-color: #fff; max-width: 300px; min-height: 50px; margin-left: auto; margin-right: auto; display: flex; flex-direction: column; justify-content: center; align-items: center; }
        .widget-button { border-radius: 4px; font-size: 14px; padding: 6px 12px; margin: 0 5px; }
        .widget-button.mod-info { background-color: #f0f8ff !important; border: 1px solid #cce5ff !important; color: #007bff !important; }
        .widget-button.mod-danger { background-color: #FF0000 !important; border: 1px solid #dee2e6 !important; color: #ffffff !important; }
        .widget-button.mod-primary { background-color: #6aa84f !important; border-color: #6aa84f !important; color: white !important; }
        .widget-button:disabled { opacity: 0.6; }
        .hint-text { font-size: 13px !important; color: #777 !important; margin-top: 25px !important; margin-bottom: 0 !important; }
        /* Adjusted analysis container style for local model */
        .local-analysis-container { text-align: left; padding: 15px; border: 2px solid #93c47d; border-radius: 8px; background-color: #fafff7; max-width: 600px; margin: 10px auto; }
    </style>
    """

    # --- 2. Create File Upload Widget ---
    uploader = widgets.FileUpload(
        accept='image/*',
        multiple=True,
        description='Choose File(s)',
        button_style='success',
        layout=widgets.Layout(width='auto', margin='10px auto 10px auto')
    )

    # --- 3. Create HTML Widgets for Content Segmentation ---
    # Header Content
    header_content = widgets.VBox([
        widgets.HTML("""<div class="camera-icon-container"><div class="camera-icon">📸</div></div>"""),
        widgets.HTML("<h3>Plant Disease Detector (Hugging Face)</h3>"),
        widgets.HTML('<p class="subtitle">Upload or capture a photo(s) of your plant to get a local AI diagnosis.</p>')
    ], layout=widgets.Layout(align_items='center'))

    # Drop Zone Content
    drop_zone_text_content = widgets.VBox([
        widgets.HTML("""<div class="upload-icon">⬆️</div>"""),
        widgets.HTML("""<div class="drop-zone-text"><p>Click to upload or drag and drop</p></div>""")
    ], layout=widgets.Layout(align_items='center', margin='0 0 10px 0'))

    # Camera Capture Button
    btn_capture = widgets.Button(
        description='Capture Photo',
        button_style='primary',
        icon='camera',
        layout=widgets.Layout(width='auto', margin='20px auto 0px auto')
    )

    # Footer Content
    footer_content = widgets.HTML(
        value="""<div class="hint-text">Using MobileNetV2 from Hugging Face for local, fast analysis.</div>"""
    )

    # Output widget for displaying the selected image
    image_display_output = widgets.Output()
    image_display_output.add_class('image-display-output')

    # Navigation Buttons
    btn_prev = widgets.Button(
        description='Previous', disabled=True, button_style='info', icon='arrow-left',
        layout=widgets.Layout(width='auto', flex='1 1 auto', margin='0 5px')
    )
    bnt_next = widgets.Button(
        description='Next', disabled=True, button_style='info', icon='arrow-right',
        layout=widgets.Layout(width='auto', flex='1 1 auto', margin='0 5px')
    )
    btn_remove = widgets.Button(
        description='Remove Photo', disabled=True, button_style='danger', icon='trash',
        layout=widgets.Layout(width='auto', flex='1 1 auto', margin='0 5px')
    )

    # Analysis Widgets
    btn_analyze = widgets.Button(
        description='✨ Analyze Photos (Local AI) ✨',
        button_style='success',
        icon='flask',
        layout=widgets.Layout(width='auto', margin='20px auto 10px auto')
    )

    analysis_output = widgets.Output()

    # --- 4. Event Handler Functions ---

    def update_image_display():
        global current_image_index, uploaded_images

        with image_display_output:
            image_display_output.clear_output(wait=True)
            if uploaded_images and 0 <= current_image_index < len(uploaded_images):
                display(Image(data=uploaded_images[current_image_index], width=200))
                display(
                    HTML(
                        f"<span style='font-size:12px; color:#777; margin-top:10px;'>Image {current_image_index + 1} of {len(uploaded_images)}</span>"
                    )
                )
            else:
                display(HTML("<span>No image selected.</span>"))

        is_image_present = bool(uploaded_images)
        btn_prev.disabled = (current_image_index == 0) or not is_image_present
        bnt_next.disabled = (current_image_index == len(uploaded_images) - 1) or not is_image_present
        btn_remove.disabled = not is_image_present

    def on_upload_change(change):
        global uploaded_images, current_image_index

        new_files_content = []
        if change.new:
            file_list = list(change.new.values())
            if file_list:
                for file_info in file_list:
                    file_content = file_info['content']
                    new_files_content.append(file_content)

        if new_files_content:
            old_length = len(uploaded_images)
            uploaded_images.extend(new_files_content)
            current_image_index = old_length
            update_image_display()
        elif not uploaded_images:
            update_image_display()

    def on_capture_click(b):
        global uploaded_images, current_image_index

        b.disabled = True
        b.description = 'Capturing...'

        captured_bytes = take_photo_to_bytes()

        b.description = 'Capture Photo'
        b.disabled = False

        if captured_bytes:
            uploaded_images.append(captured_bytes)
            current_image_index = len(uploaded_images) - 1
            update_image_display()

    def on_prev_click(b):
        global current_image_index
        if current_image_index > 0:
            current_image_index -= 1
            update_image_display()

    def on_next_click(b):
        global current_image_index
        if current_image_index < len(uploaded_images) - 1:
            current_image_index += 1
            update_image_display()

    def on_remove_click(b):
        global uploaded_images, current_image_index
        if uploaded_images and 0 <= current_image_index < len(uploaded_images):
            del uploaded_images[current_image_index]

            if uploaded_images:
                if current_image_index >= len(uploaded_images):
                    current_image_index = len(uploaded_images) - 1
            else:
                current_image_index = 0

            update_image_display()

    def on_analyze_click(b):
        global uploaded_images

        # --- 1. Clear the entire analysis_output just ONCE at the start ---
        with analysis_output:
            clear_output(wait=True)

            if not uploaded_images:
                display(HTML("<h2>🚫 Error: Please upload or capture an image first.</h2>"))
                return

            display(HTML(f"<h2>🔬 Analyzing {len(uploaded_images)} Images with Local AI:</h2>"))

        for i, img_bytes in enumerate(uploaded_images):

            loading_status_output = widgets.Output()

            with analysis_output:
                # Display image and title directly into the main output
                display(HTML(f"<h3>Image {i+1} of {len(uploaded_images)}:</h3>"))
                display(Image(data=img_bytes, width=200))

                # Display the loading message placeholder
                display(loading_status_output)

            with loading_status_output:
                # 2. Display the loading message into the temporary widget
                loading_message = HTML(
                    "<h4 style='color: #6aa84f; margin-top: 15px;'>"
                    "⏳ Analyzing with MobileNet... Please wait."
                    "</h4>"
                )
                display(loading_message)

                # --- Unified Analysis (Now just MobileNet) ---
                result_content, mode = analyze_image_unified(img_bytes)

            # 3. Clear the loading message, then display the final results below the image
            with loading_status_output:
                clear_output(wait=True)

                # --- Styling for LOCAL_HF Mode ---
                if mode == "LOCAL_HF":
                    mode_tag = "<span style='color: #38761d; font-weight: bold;'>🖥️ MobileNet Local AI (Hugging Face)</span>"
                    bg_color = "#fafff7"
                    txt_color = "#000000"
                else: # ERROR
                    mode_tag = "<span style='color: red; font-weight: bold;'>⛔ ERROR</span>"
                    bg_color = "#ffebeb"
                    txt_color = "#000000"

                # 4. Display Final Results and Separator
                display(HTML(f"<h4>Analysis Source: {mode_tag}</h4>"))
                # Use the dedicated local-analysis-container class
                display(HTML(f"<div class='local-analysis-container'><div style='background-color: {bg_color}; color: {txt_color}'>{result_content}</div></div>"))

            with analysis_output:
                # Add a separator in the main output after the results for the current image
                display(HTML("<hr>"))

    # --- Attach Event Handlers ---
    uploader.observe(on_upload_change, names='value')
    btn_capture.on_click(on_capture_click)
    btn_prev.on_click(on_prev_click)
    bnt_next.on_click(on_next_click)
    btn_remove.on_click(on_remove_click)
    btn_analyze.on_click(on_analyze_click)


    # ============================================================================
    # PART 5: ASSEMBLE AND RETURN UI
    # ============================================================================

    # Combine drop zone elements
    drop_zone_elements = widgets.VBox(
        [drop_zone_text_content, uploader, btn_capture],
        layout=widgets.Layout(width='90%', align_items='center')
    )
    drop_zone_elements.add_class('drop-zone-vbox')

    # Navigation buttons HBox
    navigation_buttons = widgets.HBox(
        [btn_prev, btn_remove, bnt_next],
        layout=widgets.Layout(
            width='auto',
            justify_content='center',
            margin='10px auto'
        )
    )

    # Main Container
    main_container_vbox = widgets.VBox(
        [
            header_content,
            drop_zone_elements,
            image_display_output,
            navigation_buttons,
            btn_analyze,
            analysis_output,
            footer_content
        ],
        layout=widgets.Layout(
            align_items='center',
            padding='20px'
        )
    )

    main_container_vbox.add_class('main-container')

    # Initialize display and button states on load
    update_image_display()

    return main_container_vbox

# Call the function to setup the UI (e.g., for use in a tabbed display)
main_container_vbox = setup_first_screen_ui()

### **Screen 2: IOT Samples**

In [24]:
import ipywidgets as widgets
import requests
import threading
import time
import datetime
from IPython.display import display

# Configuration
BASE_URL = "https://server-cloud-v645.onrender.com/"

# Shared CSS
shared_style = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

    .screen-wrapper {
        font-family: 'Roboto', sans-serif;
        background-color: #f8f9fa;
        padding: 20px;
        border-radius: 15px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.05);
        width: 98%;
        border: 1px solid #e9ecef;
        text-align: center;
        margin-bottom: 20px;
    }
    .header-title {
        color: #2c3e50;
        margin-bottom: 25px;
        font-weight: 700;
        font-size: 26px;
        text-transform: uppercase;
        letter-spacing: 1.5px;
    }
    .widget-button {
        font-size: 18px !important;
        border-radius: 10px !important;
        font-weight: bold !important;
    }
    .no-click { pointer-events: none !important; }

    /* Pulse animation for the Live badge */
    @keyframes pulse-green {
        0% { transform: scale(0.95); box-shadow: 0 0 0 0 rgba(46, 204, 113, 0.7); }
        70% { transform: scale(1.0); box-shadow: 0 0 0 10px rgba(46, 204, 113, 0); }
        100% { transform: scale(0.95); box-shadow: 0 0 0 0 rgba(46, 204, 113, 0); }
    }
    .live-badge {
        background-color: #2ecc71;
        color: white;
        padding: 5px 15px;
        border-radius: 20px;
        font-weight: bold;
        font-size: 14px;
        animation: pulse-green 2s infinite;
        display: inline-block;
        margin-bottom: 15px;
    }
</style>
"""

def fetch_iot_data(feed, limit=1):
    """Fetches data from Render server."""
    try:
        response = requests.get(f"{BASE_URL}/history", params={"feed": feed, "limit": limit}, timeout=5)
        data = response.json()
        if "data" in data:
            values = [float(x['value']) for x in data['data']]
            if limit > 1:
                values.reverse()
                return values
            else:
                return str(values[0])
    except:
        return [] if limit > 1 else "--"
    return [] if limit > 1 else "--"

def create_screen_2():
    """Builds the Live Monitor Screen with Auto-Refresh."""

    # UI Elements
    header = widgets.HTML(f"{shared_style}<div class='screen-wrapper'><div class='header-title'>📡 Real-Time Sensor Monitor</div>")

    # Status Indicator
    lbl_status = widgets.HTML(
        value="<div class='live-badge'>⚡ LIVE: Auto-updating every 10s</div><br><span style='color:#7f8c8d; font-size:12px;'>Waiting for data...</span>"
    )

    footer = widgets.HTML("</div>")

    # Display Cards (Read-only)
    card_layout = widgets.Layout(width='32%', height='100px')

    btn_temp = widgets.Button(description="Temp: --°C", button_style='danger', layout=card_layout, icon='thermometer-half')
    btn_temp.add_class('no-click')

    btn_humid = widgets.Button(description="Humid: --%", button_style='info', layout=card_layout, icon='tint')
    btn_humid.add_class('no-click')

    btn_soil = widgets.Button(description="Soil: --%", button_style='success', layout=card_layout, icon='leaf')
    btn_soil.add_class('no-click')

    # Background Thread Function
    def auto_refresh_loop():
        while True:
            # 1. Fetch Data
            t = fetch_iot_data('temperature', 1)
            h = fetch_iot_data('humidity', 1)
            s = fetch_iot_data('soil', 1)

            # 2. Update Widgets
            btn_temp.description = f" Temp: {t}°C"
            btn_humid.description = f" Humid: {h}%"
            btn_soil.description = f" Soil: {s}%"

            # Update timestamp
            now = datetime.datetime.now().strftime("%H:%M:%S")
            lbl_status.value = f"<div class='live-badge'>⚡ LIVE: Auto-updating every 10s</div><br><span style='color:#7f8c8d; font-size:12px;'>Last updated: {now}</span>"

            # 3. Wait 10 seconds
            time.sleep(10)

    # Start the thread
    # 'daemon=True' ensures the thread dies when the notebook kernel shuts down
    thread = threading.Thread(target=auto_refresh_loop, daemon=True)
    thread.start()

    # Layout Assembly
    cards_row = widgets.HBox([btn_temp, btn_humid, btn_soil], layout=widgets.Layout(justify_content='space-between', margin='10px 0'))

    # Removed the button, kept the status label
    return widgets.VBox([header, lbl_status, cards_row, footer])

In [25]:
# # Screen 2 - Live Monitor & Global Functions

# import ipywidgets as widgets
# import requests
# import matplotlib.pyplot as plt # Needed globally for both screens
# from IPython.display import display, clear_output

# # A. Global Configuration & Functions (Used by Screen 4 too)
# BASE_URL = "https://server-cloud-v645.onrender.com/"

# def fetch_latest_value(feed_name):
#     """Fetches single latest value from server."""
#     try:
#         response = requests.get(f"{BASE_URL}/history", params={"feed": feed_name, "limit": 1}, timeout=3)
#         data = response.json()
#         if "data" in data and len(data["data"]) > 0:
#             return str(data["data"][0]["value"])
#     except:
#         return "--"
#     return "--"

# def fetch_history_data(feed_name, limit_count):
#     """Fetches history list and reverses it (Oldest -> Newest)."""
#     try:
#         response = requests.get(f"{BASE_URL}/history", params={"feed": feed_name, "limit": limit_count}, timeout=5)
#         data = response.json()
#         if "data" in data:
#             vals = [float(x['value']) for x in data['data']]
#             vals.reverse() # Reverse to show chronology correctly
#             return vals
#     except:
#         return []
#     return []

# # --- B. Styling (CSS) ---
# # Added '.no-click' class to disable button interaction while keeping the look
# style_css = """
# <style>
#     @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

#     .screen-container {
#         font-family: 'Roboto', sans-serif;
#         background-color: #f4f6f7;
#         padding: 20px;
#         border-radius: 15px;
#         box-shadow: 0 4px 15px rgba(0,0,0,0.1);
#         width: 98%;
#         border: 1px solid #dcdcdc;
#         text-align: center;
#     }
#     .header-text {
#         color: #2c3e50;
#         margin-bottom: 25px;
#         font-weight: 700;
#         font-size: 28px;
#     }
#     .widget-button {
#         font-size: 20px !important;
#         border-radius: 12px !important;
#         font-weight: bold !important;
#     }
#     /* This disables clicking on the display cards */
#     .no-click {
#         pointer-events: none !important;
#     }
# </style>
# """

# # --- C. Screen 2 UI Components ---

# header = widgets.HTML(f"{style_css}<div class='screen-container'><div class='header-text'>📡 Real-Time Plant Monitor</div>")
# footer = widgets.HTML("</div>")

# # Display Buttons (Added 'no-click' class via add_class)
# btn_temp = widgets.Button(description="Temp: --°C", button_style='danger', layout=widgets.Layout(width='32%', height='100px'), icon='thermometer-half')
# btn_temp.add_class('no-click') # <--- Makes it unclickable

# btn_humid = widgets.Button(description="Humid: --%", button_style='info', layout=widgets.Layout(width='32%', height='100px'), icon='tint')
# btn_humid.add_class('no-click') # <--- Makes it unclickable

# btn_soil = widgets.Button(description="Soil: --%", button_style='success', layout=widgets.Layout(width='32%', height='100px'), icon='leaf')
# btn_soil.add_class('no-click') # <--- Makes it unclickable

# # Action Button (Clickable)
# btn_refresh = widgets.Button(
#     description=' Update Live Data',
#     icon='refresh',
#     button_style='primary',
#     layout=widgets.Layout(width='50%', height='60px', margin='20px 0 0 0')
# )

# # --- D. Logic ---

# def update_live_display(b):
#     """Fetch latest data for all sensors"""
#     btn_refresh.icon = 'spin fa-spinner'
#     btn_refresh.description = ' Updating...'

#     # Update all 3 buttons
#     t = fetch_latest_value('temperature')
#     h = fetch_latest_value('humidity')
#     s = fetch_latest_value('soil')

#     btn_temp.description = f" Temp: {t}°C"
#     btn_humid.description = f" Humid: {h}%"
#     btn_soil.description = f" Soil: {s}%"

#     btn_refresh.icon = 'refresh'
#     btn_refresh.description = ' Update Live Data'

# # Connect Event
# btn_refresh.on_click(update_live_display)

# # --- E. Layout ---
# metrics_row = widgets.HBox([btn_temp, btn_humid, btn_soil], layout=widgets.Layout(justify_content='space-between'))
# button_row = widgets.VBox([btn_refresh], layout=widgets.Layout(align_items='center', width='100%'))

# ui_screen2 = widgets.VBox([header, metrics_row, button_row, footer])

# display(ui_screen2)

# # Initial Load
# update_live_display(None)

model.safetensors:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

### **Screen 3: RAG Search Engine**

In [92]:
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output
import time

In [93]:
# =========================
# Cell 3: UI Widgets
# =========================

title_html = widgets.HTML("<h2>🤖 Ecological RAG Query</h2>")
description_html = widgets.HTML(
    "<p>Ask questions based on indexed academic papers.</p>"
)

user_query_input = widgets.Textarea(
    description='Question:',
    placeholder='e.g., What accuracy was achieved for tomato disease detection?',
    layout=widgets.Layout(width='600px', height='150px')
)

top_k_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    description='Top K docs:'
)

submit_button = widgets.Button(
    description='Generate Answer',
    button_style='success'
)

status_output = widgets.Output()
answer_output = widgets.Output()
details_output = widgets.Output()


In [94]:
# =========================
# Cell 4: Event Handler
# =========================

def display_initial_status():
    with status_output:
        clear_output()
        display(HTML("<p style='color:green;font-weight:bold;'>✅ Gemini Connected</p>"))

def on_submit_click(b):
    submit_button.disabled = True
    query = user_query_input.value.strip()

    if not query:
        with answer_output:
            clear_output()
            display(HTML("<p style='color:orange;'>Please enter a question.</p>"))
        submit_button.disabled = False
        return

    with answer_output:
        clear_output()
        display(HTML("<h3>✨ Generated Answer</h3>"))

    # Call rag_answer, which now consistently returns a dictionary
    result = rag_answer(
        query=query,
        documents=documents,
        index_urls=index_urls,
        urls=urls,
        top_k=top_k_slider.value,
        stream=True
    )

    # Check for an error condition first
    if result["error"]:
        with answer_output:
            # Display the error message from 'answer' key in the result dictionary
            display(HTML(f"<p style='color:red;'>{result['answer']}</p>"))
        submit_button.disabled = False
        return

    # If no error, proceed with streaming from the 'stream' key
    stream_box = widgets.Output()
    with answer_output: # Ensure stream_box is displayed within answer_output
        display(stream_box)

    text = ""
    for chunk in result["stream"]: # Access the generator via the 'stream' key
        if chunk:
            text += chunk
            with stream_box:
                clear_output(wait=True)
                display(Markdown(text))

    # --- Details Panel ---
    with details_output:
        clear_output()
        display(HTML("<h3>ℹ️ RAG Details</h3>"))

        for i, (url, score) in enumerate(rag_ranked_urls_list):
            display(HTML(f"{i+1}. <code>{score:.4f}</code> — {url}"))

        display(widgets.Textarea(
            value=rag_context_content,
            layout=widgets.Layout(width='100%', height='300px'),
            disabled=True
        ))

    submit_button.disabled = False

submit_button.on_click(on_submit_click)

In [95]:
# =========================
# Cell 5: CSS Styling
# =========================

query_css = """
<style>
/* --- 1. GENERAL CONTAINER & DARK MODE FIXES --- */
.rag-main-container {
    background-color: #ffffff !important;
    border: 1px solid #e9ecef !important;
    border-radius: 12px;
    padding: 30px;
    margin: 20px auto;
    font-family: 'Roboto', sans-serif;
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
    max-width: 900px;
}

/* Force dark readable text inside container */
.rag-main-container *:not(.widget-button *) {
    color: #343a40 !important;
    font-size: initial !important;
}

/* Headings & description */
.rag-main-container h2 {
    color: #007bff !important;
    font-weight: 700 !important;
}
.rag-main-container p {
    color: #6c757d !important;
    font-size: 14px !important;
}

/* --- 2. INPUT & CONTROLS --- */

/* Textarea */
.widget-textarea textarea {
    background-color: #ffffff !important;
    color: #212529 !important;
    border: 1px solid #ced4da !important;
    border-radius: 8px !important;
    padding: 10px !important;
    font-size: 16px !important;
}

/* Placeholder */
.widget-textarea textarea::placeholder {
    color: #adb5bd !important;
    opacity: 1 !important;
}

/* Slider label */
.widget-label {
    color: #495057 !important;
    font-weight: 500 !important;
}

/* Submit button */
.widget-button.mod-success {
    background-color: #28a745 !important;
    border-color: #28a745 !important;
    color: white !important;
    font-size: 18px !important;
    font-weight: bold !important;
    border-radius: 8px !important;
}
.widget-button.mod-success:hover:not(:disabled) {
    background-color: #218838 !important;
    border-color: #1e7e34 !important;
}
.widget-button:disabled {
    opacity: 0.7;
}

/* --- 3. STATUS & OUTPUTS --- */

/* Status messages */
.rag-main-container .widget-output .widget-html p[style*="color:green"] {
    color: #28a745 !important;
    font-weight: bold !important;
}
.rag-main-container .widget-output .widget-html p[style*="color:red"] {
    color: #dc3545 !important;
    font-weight: bold !important;
}

/* Generated answer box */
.rag-main-container .widget-output h3 + div {
    background-color: #f8f9fa !important;
    border: 1px solid #cce5ff !important;
    padding: 15px;
    border-radius: 5px;
}
.rag-main-container .widget-output h3 + div p {
    color: #212529 !important;
    font-size: 16px !important;
}

/* Context textarea (disabled) */
.widget-textarea.jp-mod-disabled textarea {
    background-color: #f8f9fa !important;
    color: #6c757d !important;
    border: 1px solid #e9ecef !important;
    font-family: monospace;
    font-size: 12px;
}

/* Separators */
.rag-main-container hr {
    border-top: 1px solid #ddd;
    margin: 25px 0;
}
.rag-main-container h3 {
    color: #343a40 !important;
    border-bottom: 2px solid #eee;
    padding-bottom: 5px;
}
</style>
"""




In [96]:
# =========================
# Cell 6: UI Assembly
# =========================
def create_screen_3():
    """Builds the RAG Search Engine Screen."""
    display_initial_status()
    query_screen_ui = widgets.VBox([
        title_html,
        description_html,
        status_output,
        user_query_input,
        top_k_slider,
        submit_button,
        answer_output,
        details_output
    ])

    query_screen_ui.add_class('rag-main-container')


    return query_screen_ui



### **Screen 4: Plant Dashboard**

In [31]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

def create_screen_4():
    """Builds the Historical Analytics Screen."""
    header = widgets.HTML(f"{shared_style}<div class='screen-wrapper'><div class='header-title'>📈 Historical Analytics</div>")
    footer = widgets.HTML("</div>")

    # Controls
    lbl_feed = widgets.HTML("<b>Parameter:</b>", layout=widgets.Layout(margin='0 10px 0 0'))
    dd_feed = widgets.Dropdown(
        options=[('Temperature', 'temperature'), ('Humidity', 'humidity'), ('Soil Moisture', 'soil')],
        value='temperature',
        layout=widgets.Layout(width='180px')
    )

    lbl_limit = widgets.HTML("<b>Samples (10-50):</b>", layout=widgets.Layout(margin='0 10px 0 20px'))
    input_limit = widgets.IntText(value=15, layout=widgets.Layout(width='70px'))

    out_graph = widgets.Output(layout=widgets.Layout(margin='20px 0 0 0'))

    # Logic
    def update_graph(change=None):
        feed = dd_feed.value
        limit = input_limit.value

        if limit < 5: limit = 5
        if limit > 50: limit = 50

        with out_graph:
            clear_output(wait=True)
            values = fetch_iot_data(feed, limit)

            if not values or values == "--":
                print("⚠️ No data available or Server is sleeping.")
                return

            plt.figure(figsize=(10, 4))
            plt.style.use('seaborn-v0_8-whitegrid')

            c_map = {'temperature': '#e74c3c', 'humidity': '#3498db', 'soil': '#2ecc71'}
            color = c_map.get(feed, '#333')

            plt.plot(values, marker='o', linestyle='-', linewidth=2, color=color, label=feed.capitalize())
            plt.fill_between(range(len(values)), values, color=color, alpha=0.1)

            plt.title(f"Trend Analysis: {feed.capitalize()}", fontsize=16, pad=15)
            plt.xlabel("Time (Oldest → Newest)", fontsize=12)
            plt.ylabel("Value", fontsize=12)
            plt.legend()
            plt.tight_layout()
            plt.show()

    # Events
    dd_feed.observe(update_graph, names='value')
    input_limit.observe(update_graph, names='value')

    # Initial draw
    update_graph(None)

    controls = widgets.HBox([lbl_feed, dd_feed, lbl_limit, input_limit], layout=widgets.Layout(justify_content='center', align_items='center'))
    return widgets.VBox([header, controls, out_graph, footer])

In [32]:
# # Screen 4 - Historical Analytics (Auto-Update) ---

# # A. UI Components

# header_s4 = widgets.HTML(f"{style_css}<div class='screen-container'><div class='header-text'>📈 Historical Analytics</div>")
# footer_s4 = widgets.HTML("</div>")

# # 1. Parameter Selection
# lbl_feed = widgets.HTML("<b>Select Parameter:</b>", layout=widgets.Layout(margin='0 10px 0 0'))
# dd_feed = widgets.Dropdown(
#     options=[('Temperature', 'temperature'), ('Humidity', 'humidity'), ('Soil Moisture', 'soil')],
#     value='temperature',
#     layout=widgets.Layout(width='200px')
# )

# # 2. Limit Selection
# lbl_limit = widgets.HTML("<b>Graph History:</b>", layout=widgets.Layout(margin='0 10px 0 20px'))
# input_limit = widgets.IntText(value=10, layout=widgets.Layout(width='80px'))

# # 3. Graph Output
# out_graph = widgets.Output(
#     layout=widgets.Layout(width='100%', display='flex', justify_content='center', margin='20px 0 0 0')
# )

# # --- B. Logic ---

# def update_graph(change=None): # Accepts 'change' argument from observers
#     """
#     Called whenever Dropdown or Limit changes.
#     Uses fetch_history_data from Block 1.
#     """
#     feed = dd_feed.value
#     limit = input_limit.value

#     # Validation
#     if limit < 5: limit = 5
#     if limit > 50: limit = 50

#     with out_graph:
#         clear_output(wait=True)
#         # REUSE: Calling function from Screen 2
#         values = fetch_history_data(feed, limit)

#         if not values:
#             print("⚠️ No data found or Server sleeping.")
#         else:
#             plt.figure(figsize=(12, 6))
#             plt.style.use('seaborn-v0_8-whitegrid')

#             # Dynamic colors
#             c_map = {'temperature': '#e74c3c', 'humidity': '#3498db', 'soil': '#2ecc71'}
#             c = c_map.get(feed, '#333')

#             plt.plot(values, marker='o', color=c, linewidth=3, label=feed.capitalize())
#             plt.fill_between(range(len(values)), values, color=c, alpha=0.1)

#             # Styling
#             plt.title(f"History Analysis (5-50 measures): {feed.capitalize()} (Last {limit})", fontsize=24, fontweight='bold', pad=20)
#             plt.xlabel("Sample Order (Oldest → Newest)", fontsize=18)
#             plt.ylabel("Value", fontsize=18)
#             plt.tick_params(labelsize=16)
#             plt.legend(fontsize=16)
#             plt.tight_layout()
#             plt.show()

# # --- C. Wiring Events (Auto-Update) ---

# # Observing changes triggers the graph update immediately
# dd_feed.observe(update_graph, names='value')
# input_limit.observe(update_graph, names='value')

# # --- D. Layout Assembly ---
# controls_row = widgets.HBox(
#     [lbl_feed, dd_feed, lbl_limit, input_limit],
#     layout=widgets.Layout(justify_content='center', align_items='center', margin='0 0 20px 0')
# )

# ui_screen4 = widgets.VBox([header_s4, controls_row, out_graph, footer_s4])

# display(ui_screen4)

# # Initial Graph Load
# update_graph(None)

### **Visualize Site**

In [97]:


# The rest of your Python code remains the same, as the fix is purely CSS-based.
# ... (Part A, B, and C of your Python code) ...
navbar_css = """
<style>
/* 1. Base Tab Bar Styling (The strip the tabs sit on) */
.lm-TabBar {
    background: #343a40 !important; /* Dark background for the navbar strip */
    border-bottom: 3px solid #007bff !important; /* Thin blue line at the bottom for emphasis */
    padding: 0 10px 0 10px !important; /* Add padding to sides */
    border-top-left-radius: 0 !important;
    border-top-right-radius: 0 !important;
    justify_content: center;
    align_items: center;
}

/* 2. Style Individual Tabs (The navbar buttons) */
.lm-TabBar-tab {
    /* Reset geometry to look like flat buttons */
    min-width: 200px !important; /* Ensure titles fit */
    max-width: 250px !important;
    height: 40px !important; /* Standard button height */

    /* Reset background and borders */
    background: transparent !important; /* Makes it look seamless with the bar */
    border: none !important;

    /* Text appearance for inactive links */
    color: #cccccc !important; /* Light gray text */
    font-size: 14px !important;
    font-weight: 500 !important;
    text-align: center !important;
    transition: color 0.2s; /* Smooth color change on hover */
}

/* 3. Style Selected (Active) Tab */
.lm-TabBar-tab.lm-mod-current {
    /* Highlight the text and use a bottom border as the active indicator */
    background: transparent !important;
    color: white !important; /* White text for active link */

    /* Blue line underneath the active tab */
    border-bottom: 3px solid #007bff !important;

    /* Ensure no other borders/shadows interfere */
    border-right: none !important;
    margin-bottom: -3px !important; /* Pulls the tab down slightly to meet the bottom border */
}

/* 4. Hover Effect (Optional but good for navigation) */
.lm-TabBar-tab:hover {
    color: white !important;
    background: #495057 !important; /* Slightly lighter dark gray on hover */
}

/* 5. Tab Text Fix (Critical for preventing truncation) */
.lm-TabBar-tabLabel {
    overflow: visible !important;
    white-space: nowrap !important;
    line-height: 40px !important; /* Vertically center the text */
}

/* 6. Fix for Content Panel Edges (Optional, cleans up the border between navbar and content) */
.lm-StackPanel {
    border-top: none !important;
}
button {
  display: flex !important;
  justify-content: center !important; /* Centers text horizontally */
  align-items: center !important;   /* Centers text vertically */
}
</style>
"""
# 1. Create the main Tab widget
main_tabs = widgets.Tab()

# 2. Assign the screen widgets to the Tab's children list
# NOTE: The variables main_container_vbox, ui, query_screen_ui, and dash
# must be defined in previous cells for this code block to run.
main_tabs.children = [
    main_container_vbox,
    create_screen_2(),
    create_screen_3(),
    create_screen_4()
]

# 3. Set the titles for each tab (Index starts at 0)
main_tabs.set_title(0, "1. Image Upload & Analysis")
main_tabs.set_title(1, "2. Sensor Data Sampling Control")
main_tabs.set_title(2, "3. RAG Search Engine")
main_tabs.set_title(3, "4. Visual Dashboard")
# Optional: If your original CSS was defined globally and needs to be printed once:
display(HTML(navbar_css))
display(HTML(css_style))
#display(HTML(dash_style))
#display(HTML(style_css))
display(HTML(query_css))
# 4. Final Display
display(main_tabs)


2
